In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,2020-12-18T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1091,182,1273,32592,...,2109,NaN,NaN,82859,1126037,755132.0,NaN,19.0,NaN,NaN
6275,2020-12-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,1009,191,1200,12319,...,3390,NaN,NaN,114760,1768982,1022788.0,NaN,9.0,Positivi diagnosticati solo con test antigenic...,NaN
6276,2020-12-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,265,43,308,3637,...,553,NaN,NaN,26909,469887,241362.0,NaN,3.0,NaN,NaN
6277,2020-12-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,70,7,77,423,...,365,NaN,NaN,6986,59599,36064.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,2020-12-18T17:00:00,19,Sicilia,1091,182,1273,32592,33865,-823,731,46885,2109,NaN,NaN,82859,1126037,755132.0,19.0,NaN,NaN
6275,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,97851,3390,NaN,NaN,114760,1768982,1022788.0,9.0,Positivi diagnosticati solo con test antigenic...,NaN
6276,2020-12-18T17:00:00,10,Umbria,265,43,308,3637,3945,-65,142,22411,553,NaN,NaN,26909,469887,241362.0,3.0,NaN,NaN
6277,2020-12-18T17:00:00,2,Valle d'Aosta,70,7,77,423,500,5,24,6121,365,NaN,NaN,6986,59599,36064.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-18', '2020-12-17')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-18T17:00:00,1,Piemonte,3516,243,3759,37541,41300,-4243,1210,139129,7461,NaN,NaN,187890,1595851,953662.0,14.0,In ottemperanza alle indicazioni ministeriali ...,In ottemperanza alle indicazioni ministeriali ...
1,2020-12-18T17:00:00,2,Valle d'Aosta,70,7,77,423,500,5,24,6121,365,NaN,NaN,6986,59599,36064.0,0.0,NaN,NaN
2,2020-12-18T17:00:00,3,Lombardia,4656,602,5258,62952,68210,169,2744,363044,24225,NaN,NaN,455479,4580347,2512713.0,22.0,NaN,NaN
3,2020-12-18T17:00:00,5,Veneto,2687,339,3026,94988,98014,2235,4211,105938,5268,NaN,NaN,209220,3082169,1152419.0,36.0,NaN,NaN
4,2020-12-18T17:00:00,6,Friuli Venezia Giulia,601,59,660,13091,13751,530,753,28409,1399,NaN,NaN,43559,854622,319134.0,6.0,per problemi tecnici i dati comunicati oggi so...,NaN
5,2020-12-18T17:00:00,7,Liguria,716,70,786,6680,7466,-163,287,46801,2754,NaN,NaN,57021,668390,310786.0,2.0,I 126843 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-18T17:00:00,8,Emilia-Romagna,2764,207,2971,59355,62326,-251,1745,82928,6950,NaN,NaN,152204,2398435,1171966.0,24.0,NaN,NaN
7,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,97851,3390,NaN,NaN,114760,1768982,1022788.0,9.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-18T17:00:00,10,Umbria,265,43,308,3637,3945,-65,142,22411,553,NaN,NaN,26909,469887,241362.0,3.0,NaN,NaN
9,2020-12-18T17:00:00,11,Marche,427,64,491,10051,10542,-290,381,24527,1459,NaN,NaN,36528,477649,280435.0,4.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-17T17:00:00,1,Piemonte,3597,240,3837,41706,45543,-2560,883,136345,7382,NaN,NaN,189270,1586358,1060337.0,4.0,NaN,NaN
1,2020-12-17T17:00:00,2,Valle d'Aosta,70,8,78,417,495,-28,19,6105,362,NaN,NaN,6962,59366,35964.0,1.0,NaN,NaN
2,2020-12-17T17:00:00,3,Lombardia,4793,611,5404,62637,68041,-1692,2730,360529,24165,NaN,NaN,452735,4546501,2500519.0,44.0,NaN,NaN
3,2020-12-17T17:00:00,5,Veneto,2701,352,3053,92726,95779,1554,4402,104069,5161,NaN,NaN,205009,3063368,1146309.0,29.0,NaN,NaN
4,2020-12-17T17:00:00,6,Friuli Venezia Giulia,630,57,687,12534,13221,-530,707,28219,1366,NaN,NaN,42806,846841,319134.0,9.0,NaN,NaN
5,2020-12-17T17:00:00,7,Liguria,746,69,815,6814,7629,-400,416,46381,2724,NaN,NaN,56734,664622,309643.0,4.0,I 123979 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-17T17:00:00,8,Emilia-Romagna,2801,206,3007,59570,62577,-269,1667,81008,6875,NaN,NaN,150460,2381673,1167304.0,6.0,NaN,NaN
7,2020-12-17T17:00:00,9,Toscana,1062,199,1261,12999,14260,-775,636,96638,3348,NaN,NaN,114246,1757598,1019118.0,16.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-17T17:00:00,10,Umbria,270,42,312,3698,4010,-313,180,22208,549,NaN,NaN,26767,467015,240515.0,2.0,NaN,NaN
9,2020-12-17T17:00:00,11,Marche,436,67,503,10329,10832,-299,467,23869,1446,NaN,NaN,36147,472564,277059.0,1.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-18T17:00:00,1,Piemonte,3516,243,3759,37541,41300,-4243,1210,139129,7461,NaN,NaN,187890,1595851,953662.0,14.0,In ottemperanza alle indicazioni ministeriali ...,In ottemperanza alle indicazioni ministeriali ...
1,2020-12-18T17:00:00,2,Valle d'Aosta,70,7,77,423,500,5,24,6121,365,NaN,NaN,6986,59599,36064.0,0.0,NaN,NaN
2,2020-12-18T17:00:00,3,Lombardia,4656,602,5258,62952,68210,169,2744,363044,24225,NaN,NaN,455479,4580347,2512713.0,22.0,NaN,NaN
3,2020-12-18T17:00:00,4,Trentino-Alto Adige,553,73,626,12977,13603,-160,420,32368,1514,0.0,0.0,47485,766655,304462.0,6.0,0,NaN
4,2020-12-18T17:00:00,5,Veneto,2687,339,3026,94988,98014,2235,4211,105938,5268,NaN,NaN,209220,3082169,1152419.0,36.0,NaN,NaN
5,2020-12-18T17:00:00,6,Friuli Venezia Giulia,601,59,660,13091,13751,530,753,28409,1399,NaN,NaN,43559,854622,319134.0,6.0,per problemi tecnici i dati comunicati oggi so...,NaN
6,2020-12-18T17:00:00,7,Liguria,716,70,786,6680,7466,-163,287,46801,2754,NaN,NaN,57021,668390,310786.0,2.0,I 126843 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-18T17:00:00,8,Emilia-Romagna,2764,207,2971,59355,62326,-251,1745,82928,6950,NaN,NaN,152204,2398435,1171966.0,24.0,NaN,NaN
8,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,97851,3390,NaN,NaN,114760,1768982,1022788.0,9.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-18T17:00:00,10,Umbria,265,43,308,3637,3945,-65,142,22411,553,NaN,NaN,26909,469887,241362.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-17T17:00:00,1,Piemonte,3597,240,3837,41706,45543,-2560,...,136345,7382,NaN,NaN,189270,1586358,1060337.0,4.0,NaN,NaN
1,1,2020-12-17T17:00:00,2,Valle d'Aosta,70,8,78,417,495,-28,...,6105,362,NaN,NaN,6962,59366,35964.0,1.0,NaN,NaN
2,2,2020-12-17T17:00:00,3,Lombardia,4793,611,5404,62637,68041,-1692,...,360529,24165,NaN,NaN,452735,4546501,2500519.0,44.0,NaN,NaN
3,19,2020-12-17T17:00:00,4,Trentino-Alto Adige,598,70,668,13095,13763,307,...,31808,1494,0.0,0.0,47065,762186,303577.0,6.0,0,NaN
4,3,2020-12-17T17:00:00,5,Veneto,2701,352,3053,92726,95779,1554,...,104069,5161,NaN,NaN,205009,3063368,1146309.0,29.0,NaN,NaN
5,4,2020-12-17T17:00:00,6,Friuli Venezia Giulia,630,57,687,12534,13221,-530,...,28219,1366,NaN,NaN,42806,846841,319134.0,9.0,NaN,NaN
6,5,2020-12-17T17:00:00,7,Liguria,746,69,815,6814,7629,-400,...,46381,2724,NaN,NaN,56734,664622,309643.0,4.0,I 123979 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-17T17:00:00,8,Emilia-Romagna,2801,206,3007,59570,62577,-269,...,81008,6875,NaN,NaN,150460,2381673,1167304.0,6.0,NaN,NaN
8,7,2020-12-17T17:00:00,9,Toscana,1062,199,1261,12999,14260,-775,...,96638,3348,NaN,NaN,114246,1757598,1019118.0,16.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-17T17:00:00,10,Umbria,270,42,312,3698,4010,-313,...,22208,549,NaN,NaN,26767,467015,240515.0,2.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-18T17:00:00,1,Piemonte,3516,243,3759,37541,41300,-4243,1210,...,7461,NaN,NaN,187890,1595851,953662.0,14.0,In ottemperanza alle indicazioni ministeriali ...,In ottemperanza alle indicazioni ministeriali ...,4341375
1,2020-12-18T17:00:00,2,Valle d'Aosta,70,7,77,423,500,5,24,...,365,NaN,NaN,6986,59599,36064.0,0.0,NaN,NaN,125501
2,2020-12-18T17:00:00,3,Lombardia,4656,602,5258,62952,68210,169,2744,...,24225,NaN,NaN,455479,4580347,2512713.0,22.0,NaN,NaN,10103969
3,2020-12-18T17:00:00,4,Trentino-Alto Adige,553,73,626,12977,13603,-160,420,...,1514,0.0,0.0,47485,766655,304462.0,6.0,0,NaN,1074819
4,2020-12-18T17:00:00,5,Veneto,2687,339,3026,94988,98014,2235,4211,...,5268,NaN,NaN,209220,3082169,1152419.0,36.0,NaN,NaN,4907704
5,2020-12-18T17:00:00,6,Friuli Venezia Giulia,601,59,660,13091,13751,530,753,...,1399,NaN,NaN,43559,854622,319134.0,6.0,per problemi tecnici i dati comunicati oggi so...,NaN,1211357
6,2020-12-18T17:00:00,7,Liguria,716,70,786,6680,7466,-163,287,...,2754,NaN,NaN,57021,668390,310786.0,2.0,I 126843 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-18T17:00:00,8,Emilia-Romagna,2764,207,2971,59355,62326,-251,1745,...,6950,NaN,NaN,152204,2398435,1171966.0,24.0,NaN,NaN,4467118
8,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,...,3390,NaN,NaN,114760,1768982,1022788.0,9.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-18T17:00:00,10,Umbria,265,43,308,3637,3945,-65,142,...,553,NaN,NaN,26909,469887,241362.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-18T17:00:00,1,Piemonte,3516,243,3759,37541,41300,-4243,1210,...,NaN,NaN,187890,1595851,953662.0,14.0,In ottemperanza alle indicazioni ministeriali ...,In ottemperanza alle indicazioni ministeriali ...,4341375,575
1,2020-12-18T17:00:00,2,Valle d'Aosta,70,7,77,423,500,5,24,...,NaN,NaN,6986,59599,36064.0,0.0,NaN,NaN,125501,20
2,2020-12-18T17:00:00,3,Lombardia,4656,602,5258,62952,68210,169,2744,...,NaN,NaN,455479,4580347,2512713.0,22.0,NaN,NaN,10103969,1036
3,2020-12-18T17:00:00,4,Trentino-Alto Adige,553,73,626,12977,13603,-160,420,...,0.0,0.0,47485,766655,304462.0,6.0,0,NaN,1074819,106
4,2020-12-18T17:00:00,5,Veneto,2687,339,3026,94988,98014,2235,4211,...,NaN,NaN,209220,3082169,1152419.0,36.0,NaN,NaN,4907704,1016
5,2020-12-18T17:00:00,6,Friuli Venezia Giulia,601,59,660,13091,13751,530,753,...,NaN,NaN,43559,854622,319134.0,6.0,per problemi tecnici i dati comunicati oggi so...,NaN,1211357,180
6,2020-12-18T17:00:00,7,Liguria,716,70,786,6680,7466,-163,287,...,NaN,NaN,57021,668390,310786.0,2.0,I 126843 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-18T17:00:00,8,Emilia-Romagna,2764,207,2971,59355,62326,-251,1745,...,NaN,NaN,152204,2398435,1171966.0,24.0,NaN,NaN,4467118,634
8,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,...,NaN,NaN,114760,1768982,1022788.0,9.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-18T17:00:00,10,Umbria,265,43,308,3637,3945,-65,142,...,NaN,NaN,26909,469887,241362.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-18T17:00:00,5,Veneto,2687,339,3026,94988,98014,2235,4211,...,33.37,2262,-191,18801,6110.0,107,1869,22.40,0.08580,4.26
1,2020-12-18T17:00:00,3,Lombardia,4656,602,5258,62952,68210,169,2744,...,58.11,315,14,33846,12194.0,60,2515,8.11,0.02716,4.51
2,2020-12-18T17:00:00,8,Emilia-Romagna,2764,207,2971,59355,62326,-251,1745,...,32.65,-215,78,16762,4662.0,75,1920,10.41,0.03906,3.41
3,2020-12-18T17:00:00,12,Lazio,2780,310,3090,74069,77159,-1471,1428,...,36.60,-1390,-169,15984,7045.0,74,2825,8.93,0.02435,2.50
4,2020-12-18T17:00:00,16,Puglia,1457,174,1631,51203,52834,218,1314,...,47.15,240,241,9346,4035.0,30,1066,14.06,0.03278,1.96
5,2020-12-18T17:00:00,1,Piemonte,3516,243,3759,37541,41300,-4243,1210,...,42.26,-4165,327,9493,-106675.0,79,2784,12.75,0.02787,4.33
6,2020-12-18T17:00:00,15,Campania,1621,120,1741,82956,84697,-1856,1201,...,23.76,-1822,274,18146,9403.0,56,3001,6.62,0.02076,3.09
7,2020-12-18T17:00:00,6,Friuli Venezia Giulia,601,59,660,13091,13751,530,753,...,32.78,557,46,7781,0.0,33,190,9.68,0.06216,3.60
8,2020-12-18T17:00:00,19,Sicilia,1091,182,1273,32592,33865,-823,731,...,30.95,-786,-141,8109,5076.0,22,1532,9.01,0.01471,1.67
9,2020-12-18T17:00:00,9,Toscana,1009,191,1200,12319,13519,-741,514,...,36.52,-680,-122,11384,3670.0,42,1213,4.52,0.01381,3.08


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,73,33,106,68.87
1,Lombardia,602,434,1036,58.11
2,Puglia,174,195,369,47.15
3,Marche,64,79,143,44.76
4,Piemonte,243,332,575,42.26
5,Umbria,43,74,117,36.75
6,Lazio,310,537,847,36.60
7,Toscana,191,332,523,36.52
8,Valle d'Aosta,7,13,20,35.00
9,Liguria,70,139,209,33.49


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2020-12-16T17:00:00,22,P.A. Trento,414,50,464,2338,2802,95,244,15480,808,NaN,NaN,19090,417683,145965.0,3.0,NaN,"Dei 244 casi odierni, 138 sono molecolari con ..."
594,2020-12-17T17:00:00,21,P.A. Bolzano,190,22,212,10500,10712,58,287,16127,673,NaN,NaN,27512,341608,156988.0,3.0,NaN,NaN
595,2020-12-17T17:00:00,22,P.A. Trento,408,48,456,2595,3051,249,463,15681,821,NaN,NaN,19553,420578,146589.0,3.0,NaN,"Dei 463 casi odierni, 348 sono molecolari con ..."
596,2020-12-18T17:00:00,21,P.A. Bolzano,175,23,198,10545,10743,31,169,16259,679,NaN,NaN,27681,342805,157406.0,3.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2020-12-14T17:00:00,4,Trentino-Alto Adige,644,79,723,12443,13166,52,182,...,25.0,1538.0,505.0,37.0,10.93,5.49,11.83,36.04,6.76,0.40
295,2020-12-15T17:00:00,4,Trentino-Alto Adige,620,77,697,12640,13337,171,465,...,22.0,2801.0,800.0,197.0,11.05,5.23,16.60,58.13,-2.53,1.30
296,2020-12-16T17:00:00,4,Trentino-Alto Adige,617,72,689,12767,13456,119,529,...,24.0,5369.0,955.0,127.0,10.45,5.12,9.85,55.39,-6.49,0.89
297,2020-12-17T17:00:00,4,Trentino-Alto Adige,598,70,668,13095,13763,307,750,...,18.0,4881.0,1238.0,328.0,10.48,4.85,15.37,60.58,-2.78,2.28


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
